In [52]:
import re
from pathlib import Path
import numpy as np

In [3]:
with open(Path("beaminput.txt"),'r') as f:
    lines = f.readlines()

In [48]:
def matched_index(lines, pattern, **kwargs):
    matched_idx = []
    if 'group' in kwargs.keys():
        re_group_num = kwargs['group']
        re_group_enable = True
    else:
        re_group_num = 1
        re_group_enable = False
    contents = []
    for i, line in enumerate(lines):        
        if re.search(pattern, line):
            matched_idx.append(i)
            if re_group_enable:
                contents.append(re.search(pattern, line).group(re_group_num))
    if len(matched_idx) == 0:
        print("No match found")
    elif len(matched_idx) == 1:
        return matched_idx[0], contents
    print("Matched indices: ", matched_idx)
    print("Matched contents: ", contents)
    return matched_idx, contents


In [58]:
matched_string = "!!number of beams "
i, _ = matched_index(lines, matched_string)
print(i)


2


In [59]:
num_beams, num_cros_prop, flag_property =tuple(map(int,lines[i+1].split('\t')))

In [56]:
matched_string = "beam (.*?):"
matched_indexes, contents = matched_index(lines, matched_string, group=1)


Matched indices:  [6, 10, 34, 57, 61, 95, 128, 132, 166, 199, 203, 237, 270, 274, 308]
Matched contents:  ['monopile', 'monopile', 'monopile', 'tower', 'tower', 'tower', 'blade 1', 'blade 1', 'blade 1', 'blade 2', 'blade 2', 'blade 2', 'blade 3', 'blade 3', 'blade 3']


In [54]:
indexes = np.unique(contents, return_index=True)[1]
beam_types = [contents[index] for index in sorted(indexes)]


['monopile', 'tower', 'blade 1', 'blade 2', 'blade 3']

In [63]:
matched_string = "beam (.*?): number of nodes"
matched_indexes, contents = matched_index(lines, matched_string, group=1)
beam_data = {}
for i, idx in enumerate(matched_indexes):
    line = lines[idx+1]
    num_nodes = int(line.split('\t')[0])
    num_elements = int(line.split('\t')[1])
    beam_data[contents[i]] = {'num_nodes': num_nodes, 'num_elements': num_elements}
print(beam_data)


Matched indices:  [6, 57, 128, 199, 270]
Matched contents:  ['monopile', 'tower', 'blade 1', 'blade 2', 'blade 3']
{'monopile': {'num_nodes': 21, 'num_elements': 20}, 'tower': {'num_nodes': 31, 'num_elements': 30}, 'blade 1': {'num_nodes': 31, 'num_elements': 30}, 'blade 2': {'num_nodes': 31, 'num_elements': 30}, 'blade 3': {'num_nodes': 31, 'num_elements': 30}}


In [64]:
matched_string = "beam (.*?): nodal phi "
matched_indexes, contents = matched_index(lines, matched_string, group=1)

Matched indices:  [10, 61, 132, 203, 274]
Matched contents:  ['monopile', 'tower', 'blade 1', 'blade 2', 'blade 3']


In [75]:
pattern = r"(-?\d+\.\d+)"
def extract_coord_d1_d2_d3(line, pattern = r"(-?\d+\.\d+)"):
    data = [float(x) for x in re.findall(pattern, line)]
    it = iter(data)
    coordinates, d1, d2, d3 = list(zip(it, it,it))
    beam_data[contents[i]]['coordinates'].append(coordinates)
    beam_data[contents[i]]['d1'].append(d1)
    beam_data[contents[i]]['d2'].append(d2)
    beam_data[contents[i]]['d3'].append(d3)
for i, idx in enumerate(matched_indexes):
    starting_idx = idx+1
    line = lines[starting_idx]
    beam_data[contents[i]]['coordinates'] = []
    beam_data[contents[i]]['d1'] = []
    beam_data[contents[i]]['d2'] = []
    beam_data[contents[i]]['d3'] = []
    extract_coord_d1_d2_d3(line, pattern)
    
    
    while True:
        
        starting_idx += 1
        line = lines[starting_idx]
        if '!' in line:
            break
        extract_coord_d1_d2_d3(line, pattern)
    

In [77]:
matched_string = "beam (.*?): connectivities "
matched_indexes, contents = matched_index(lines, matched_string, group=1)

Matched indices:  [34, 95, 166, 237, 308]
Matched contents:  ['monopile', 'tower', 'blade 1', 'blade 2', 'blade 3']


In [78]:
for i, idx in enumerate(matched_indexes):
    starting_idx = idx+1
    line = lines[starting_idx]
    beam_data[contents[i]]['connectivities'] = []
    beam_data[contents[i]]['cross_prop'] = []
    n1,n2, cross_prop = tuple(map(int,line.split('\t')))
    beam_data[contents[i]]['connectivities'].append((n1,n2))
    beam_data[contents[i]]['cross_prop'].append(cross_prop)
    while True:
        starting_idx += 1
        line = lines[starting_idx]
        if '!' in line:
            break
        n1,n2, cross_prop = tuple(map(int,line.split('\t')))
        beam_data[contents[i]]['connectivities'].append((n1,n2))
        beam_data[contents[i]]['cross_prop'].append(cross_prop)

In [79]:
beam_data

{'monopile': {'num_nodes': 21,
  'num_elements': 20,
  'coordinates': [(0.0, 0.0, -30.0),
   (0.0, 0.0, -27.74977497749775),
   (0.0, 0.0, -25.4995499549955),
   (0.0, 0.0, -23.24932493249325),
   (0.0, 0.0, -20.999099909991),
   (0.0, 0.0, -18.74887488748875),
   (0.0, 0.0, -16.498649864986497),
   (0.0, 0.0, -14.248424842484248),
   (0.0, 0.0, -11.998199819981998),
   (0.0, 0.0, -9.7480881691408),
   (0.0, 0.0, -7.498875393612236),
   (0.0, 0.0, -5.249662618083669),
   (0.0, 0.0, -3.000449842555106),
   (0.0, 0.0, -0.751237067026544),
   (0.0, 0.0, 1.498649864986498),
   (0.0, 0.0, 3.74887488748875),
   (0.0, 0.0, 5.999099909991002),
   (0.0, 0.0, 8.249324932493248),
   (0.0, 0.0, 10.4995499549955),
   (0.0, 0.0, 12.749774977497747),
   (0.0, 0.0, 15.0)],
  'd1': [(1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.0, 0.0, 0.0),
   (1.